# Agentic Workflows with CrewAI Flows

CrewAI Flows is a feature designed to streamline the creation and management of AI workflows. Flows allow developers to combine and coordinate coding tasks and Crews efficiently, providing a robust framework for building sophisticated AI automations. You can start easily by leveraging two decorators, `@start()` and `@listen()`:

- `@start()`: the `@start()` decorator is used to mark a method as the starting point of a Flow; when a Flow is started, all the methods decorated with `@start()` are executed in parallel. You can have multiple start methods in a Flow, and they will all be executed when the Flow is started.
- `@listen()`: the `@listen()` decorator is used to mark a method as a listener for the output of another task in the Flow. The method decorated with `@listen()` will be executed when the specified task emits an output. The method can access the output of the task it is listening to as an argument. The `@listen()` decorator can be used in several ways:
    - Listening to a Method by Name: You can pass the name of the method you want to listen to as a string. When that method completes, the listener method will be triggered.
    - Listening to a Method Directly: You can pass the method itself. When that method completes, the listener method will be triggered.​

In [ ]:
%pip install -r crewai-requirements.txt --quiet --upgrade

In [ ]:
import nest_asyncio
nest_asyncio.apply()

Start by configuring the LLM.

In [ ]:
from crewai import LLM

llm = LLM(
    model="bedrock/us.amazon.nova-pro-v1:0",    # Use Amazon Bedrock models 
    # model="sagemaker/INSERT ENDPOINT NAME",     # Use Amazon SageMaker AI Inference
    temperature=0.7, max_tokens=4*1024,
)

In [ ]:
llm.call("What is the capital of France?")

Now, set up the agent.

In [ ]:
from dotenv import load_dotenv
from crewai_tools import SerperDevTool
from crewai.tools import tool
from duckduckgo_search import DDGS
import os

load_dotenv()

@tool('DuckDuckGoSearch')
def search_with_duckduckgo(search_query: str):
    """Search the web for information on a given topic"""
    return DDGS().text(search_query, max_results=5)

if os.environ.get("SERPER_API_KEY"):
    search_tool = SerperDevTool()
    print("Using Serper API for search")
else:
    search_tool = search_with_duckduckgo
    print("No Serper API Key found - Using DuckDuckGo API for search")

In [ ]:
from crewai import Agent
from textwrap import dedent


researcher_agent = Agent(
    role="Travel Researcher",
    goal="Research and compile interesting activities and attractions for a given location",
    backstory=dedent(
        """You are an experienced travel researcher with a knack for 
        discovering both popular attractions and hidden gems in any 
        location. Your expertise lies in gathering comprehensive 
        information about various activities, their historical 
        significance, and practical details for visitors.
        """),
    llm=llm,
    allow_delegation=False, max_iter=4,
    tools=[search_tool],
    verbose=True,
)

content_writer = Agent(
    role="Content Writer",
    goal="Write a listicle of 5+ attractions/activities for a given location",
    backstory=dedent(
        """You are a content writer with a knack for creating engaging
        and informative content for travel blogs. Your expertise lies in
        crafting engaging and informative content for travel blogs.
        """),
    llm=llm,
    allow_delegation=False, max_iter=4,
    verbose=True,
)

editor_agent = Agent(
    role="Content Editor",
    goal="Ensure the listicle is well-structured, engaging, and error-free",
    backstory=dedent(
        """You are a meticulous editor with years of experience in
        travel content. Your keen eye for detail helps polish articles
        to perfection. You focus on improving flow, maintaining
        consistency, and enhancing the overall readability of the
        content while ensuring it appeals to the target audience.
        """),
    llm=llm,
    allow_delegation=False, max_iter=4,
    verbose=True,
)

Create the flow:

1. Search online information about the city to visit
2. Write the content in the form of a listicle
3. Review and edit the content

In [ ]:
from crewai.flow.flow import Flow, listen, start
from pydantic import BaseModel


class TravelAgentFlow(Flow[str]):
    @start()
    async def search_online(self):
        query = "Best things to do in {self.city}"
        result = await researcher_agent.kickoff_async(query)
        return result

    @listen(search_online)
    async def write_content(self, search_result):
        query = "{search_result}\n\n Based on the search results, write a listicle of 5 things to do in {self.city}"
        result = await content_writer.kickoff_async(query)
        return result
    
    @listen(write_content)
    async def edit_content(self, listicle):
        query = "Review and edit the top 5 listicle article about things to do in {self.city}.\n\nContent:\n{listicle}\n\nMake sure the content is well-structured, engaging, and error-free."
        result = await editor_agent.kickoff_async(query)
        return result

# Run the flow
flow = TravelAgentFlow()
final_output = await flow.kickoff_async({"city": "Paris"})
print("---- Final Output ----")
print(final_output)